**url:**

https://sputnikvaccine.com/newsroom/?PAGEN_1=5

In [1]:
import pandas as pd,requests,bs4,re,time
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
SITE_NAME='Sputnikvaccine'

DOMAIN = "https://sputnikvaccine.com"

SITE_LINK="https://sputnikvaccine.com/newsroom"

In [3]:
def remove_esc_chars(text):
    return text.replace("\n", " ").replace("\t", " ").replace("\r", " ")

In [4]:
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [5]:
def get_news_data(element):
    titles,published_dates,thumbnails,hrefs,authors,articles = [],[],[],[],[],[]
    #titles
    headlines = element.find('a')
    for t in headlines:
        if t != None:
            title = remove_esc_chars(t.text)
            titles.append(title.strip())
        else:
            titles.append("")

    #dates
    dates = element.find_all('span',{'class' :'newsroom-template__item-date'})
    for d in dates:
        if d != None:
            published_date = d.text.replace(",","")
            published_dates.append(published_date.strip())
        else:
            published_dates.append("")


    #thumbnails
    T = 'https://sputnikvaccine.com/local/templates/sputnik/img/logo_inner_new.svg'
    thumbnails.append(T.strip())

    site_all_hrefs = []
    #hrefs and authors
    hrefs_container = element.find_all('a')
    for href in hrefs_container:
        if href != None:
            href = href['href']
            hrefs.append(href.strip())
            site_all_hrefs.append(href.strip())
            authors.append(SITE_NAME.strip())
        else:
            hrefs.append("")
            authors.append(SITE_NAME.strip())

    #extracting text 
    for number,link in enumerate (site_all_hrefs):
        try:
            url = site_all_hrefs[number]
            g = Goose()
            article_extract = g.extract(url=url)
            article = remove_esc_chars(article_extract.cleaned_text)
            meta_data = remove_esc_chars(article_extract.meta_description)
            whole_data = meta_data+article
            if re.findall('Your browser will redirect to your requested content shortly',whole_data):
                articles.append("")
            else:
                articles.append(whole_data)
        except:
            articles.append("")
    return titles,published_dates,hrefs,authors,thumbnails,articles

In [6]:
article_list = []
pagination = 1
last_page = 10 #10
while True:
    url = f"https://sputnikvaccine.com/newsroom/?PAGEN_1={pagination}"
    soup = parse_webpage_bs(url)
    if soup != None:
        elements = soup.find_all('div',{'class' :'newsroom-template__item'})
        for element in elements: 
            titles,published_dates,links,authors,thumbnails,texts = get_news_data(element)
            datas = [titles,published_dates,links,authors,thumbnails,texts]
            
            for x in range(len(datas[0])):
                try:
                    published_date = datas[datas.index(published_dates)][x]
                    title = datas[datas.index(titles)][x]
                    text = datas[datas.index(texts)][x]
                    link = datas[datas.index(links)][x]
                    thumbnail = datas[datas.index(thumbnails)][x]
                    author = datas[datas.index(authors)][x]
                    article = (published_date,title,text,link,thumbnail,author)
                    article_list.append(article)
                    print(published_date,title)
                except:
                    pass
                
    pagination += 1
    
    print(str(last_page-pagination+1) + " pages remaining..",end="\r")
    
    if pagination > last_page:
        break   

06.02.2022 Russia says India approves one-shot Sputnik Light COVID vaccine
06.02.2022 Sputnik Light vaccine gets EUA in India
20.01.2022 COVID-19: Sputnik V shows strong protection against Omicron compared to Pfizer vaccine, says Italy-Russia joint study
20.01.2022 Sputnik V better than Comirnaty against Omicron, study says
25.12.2021 Laos becomes 20th country to authorise one-shot Sputnik Light vaccine
25.11.2021 Sputnik V shows 80 long-term efficacy against COVID in San Marino from 6th through 8th month after administering the second dose
25.11.2021 RDIF: Sputnik most efficient jab in study in Hungary
25.11.2021 Russia says Sputnik gives longer protection than Western coronavirus jabs
25.11.2021 Russia says Sputnik gives longer protection than Western jabs
03.11.2021 Sputnik Light produces strong level of antibodies against COVID-19 - early-stage trial
01.11.2021 Argentinian study proves high efficacy of Sputnik V in protecting people over 60: RDIF
31.10.2021 Russia’s Sputnik V, Sput

In [7]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df

,date,title,article,url,thumbnail,author
0,06.02.2022,Russia says India approves one-shot Sputnik Li...,India has given regulatory approval to Russia'...,https://www.reuters.com/world/india/russia-say...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
1,06.02.2022,Sputnik Light vaccine gets EUA in India,Sputnik Light has been authorized in more than...,https://health.economictimes.indiatimes.com/ne...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
2,20.01.2022,COVID-19: Sputnik V shows strong protection ag...,"New Delhi [India], January 20 (ANI): Russia's ...",https://www.aninews.in/news/national/general-n...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
3,20.01.2022,Sputnik V better than Comirnaty against Omicro...,,https://www.thepharmaletter.com/article/sputni...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
4,25.12.2021,Laos becomes 20th country to authorise one-sho...,Laos becomes 20th country to authorise one-sho...,https://asianews.network/laos-becomes-20th-cou...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
...,...,...,...,...,...,...
95,04.03.2021,Russia’s Sputnik V vaccine authorized in Iraq,,https://tass.com/economy/1262891,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
96,04.03.2021,Russian Sputnik V vaccine registered in Laos,,https://tass.com/economy/1262855,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
97,04.03.2021,Sri Lanka approves Russia's Sputnik V vaccine ...,Sri Lanka has approved Russia's Sputnik V vacc...,https://www.reuters.com/article/us-health-coro...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine
98,04.03.2021,Europe starts formal review of Russia's Sputni...,Europe's medicines regulator said on Thursday ...,https://www.reuters.com/article/us-health-coro...,https://sputnikvaccine.com/local/templates/spu...,Sputnikvaccine


In [8]:
#to csv
temp_df.to_csv('Sputnikvaccine news.csv')

#to json
temp_df.to_json('Sputnikvaccine news.json')